In [30]:
import pandas as pd
import numpy as np

In [5]:
House_df=pd.read_csv("HousePricePrediction.csv")

In [6]:
House_df

,Id,MSSubClass,MSZoning,LotArea,LotConfig,BldgType,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,BsmtFinSF2,TotalBsmtSF,SalePrice
0,0,60,RL,8450,Inside,1Fam,5,2003,2003,VinylSd,0.0,856.0,208500.0
1,1,20,RL,9600,FR2,1Fam,8,1976,1976,MetalSd,0.0,1262.0,181500.0
2,2,60,RL,11250,Inside,1Fam,5,2001,2002,VinylSd,0.0,920.0,223500.0
3,3,70,RL,9550,Corner,1Fam,5,1915,1970,Wd Sdng,0.0,756.0,140000.0
4,4,60,RL,14260,FR2,1Fam,5,2000,2000,VinylSd,0.0,1145.0,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2914,160,RM,1936,Inside,Twnhs,7,1970,1970,CemntBd,0.0,546.0,NaN
2915,2915,160,RM,1894,Inside,TwnhsE,5,1970,1970,CemntBd,0.0,546.0,NaN
2916,2916,20,RL,20000,Inside,1Fam,7,1960,1996,VinylSd,0.0,1224.0,NaN
2917,2917,85,RL,10441,Inside,1Fam,5,1992,1992,HdBoard,0.0,912.0,NaN


In [ ]:


#DATA CLEANING AND FEATURE ENGINEERING



In [15]:
#Separate training data because some data contains NaN value in price 
 #and the model cannot learn without target value so we will use those
  #data as training data

train_df=House_df[House_df['SalePrice'].notnull()]
 # OR  train_df = House_df.dropna(subset=['SalePrice'])

test_df=House_df[House_df['SalePrice'].isnull()]
train_df

,Id,MSSubClass,MSZoning,LotArea,LotConfig,BldgType,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,BsmtFinSF2,TotalBsmtSF,SalePrice
0,0,60,RL,8450,Inside,1Fam,5,2003,2003,VinylSd,0.0,856.0,208500.0
1,1,20,RL,9600,FR2,1Fam,8,1976,1976,MetalSd,0.0,1262.0,181500.0
2,2,60,RL,11250,Inside,1Fam,5,2001,2002,VinylSd,0.0,920.0,223500.0
3,3,70,RL,9550,Corner,1Fam,5,1915,1970,Wd Sdng,0.0,756.0,140000.0
4,4,60,RL,14260,FR2,1Fam,5,2000,2000,VinylSd,0.0,1145.0,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,60,RL,7917,Inside,1Fam,5,1999,2000,VinylSd,0.0,953.0,175000.0
1456,1456,20,RL,13175,Inside,1Fam,6,1978,1988,Plywood,163.0,1542.0,210000.0
1457,1457,70,RL,9042,Inside,1Fam,9,1941,2006,CemntBd,0.0,1152.0,266500.0
1458,1458,20,RL,9717,Inside,1Fam,6,1950,1996,MetalSd,1029.0,1078.0,142125.0


In [16]:


test_df



,Id,MSSubClass,MSZoning,LotArea,LotConfig,BldgType,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,BsmtFinSF2,TotalBsmtSF,SalePrice
1460,1460,20,RH,11622,Inside,1Fam,6,1961,1961,VinylSd,144.0,882.0,NaN
1461,1461,20,RL,14267,Corner,1Fam,6,1958,1958,Wd Sdng,0.0,1329.0,NaN
1462,1462,60,RL,13830,Inside,1Fam,5,1997,1998,VinylSd,0.0,928.0,NaN
1463,1463,60,RL,9978,Inside,1Fam,6,1998,1998,VinylSd,0.0,926.0,NaN
1464,1464,120,RL,5005,Inside,TwnhsE,5,1992,1992,HdBoard,0.0,1280.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,2914,160,RM,1936,Inside,Twnhs,7,1970,1970,CemntBd,0.0,546.0,NaN
2915,2915,160,RM,1894,Inside,TwnhsE,5,1970,1970,CemntBd,0.0,546.0,NaN
2916,2916,20,RL,20000,Inside,1Fam,7,1960,1996,VinylSd,0.0,1224.0,NaN
2917,2917,85,RL,10441,Inside,1Fam,5,1992,1992,HdBoard,0.0,912.0,NaN


In [17]:
#Drop Id
 # Id is just an identifier
 # It has no relationship with house price
 # Including it introduces noise

train_df=train_df.drop(columns=['Id'])
 #OR  train_df=train_df.drop(['Id'], axis=1)

train_df

,MSSubClass,MSZoning,LotArea,LotConfig,BldgType,OverallCond,YearBuilt,YearRemodAdd,Exterior1st,BsmtFinSF2,TotalBsmtSF,SalePrice
0,60,RL,8450,Inside,1Fam,5,2003,2003,VinylSd,0.0,856.0,208500.0
1,20,RL,9600,FR2,1Fam,8,1976,1976,MetalSd,0.0,1262.0,181500.0
2,60,RL,11250,Inside,1Fam,5,2001,2002,VinylSd,0.0,920.0,223500.0
3,70,RL,9550,Corner,1Fam,5,1915,1970,Wd Sdng,0.0,756.0,140000.0
4,60,RL,14260,FR2,1Fam,5,2000,2000,VinylSd,0.0,1145.0,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,7917,Inside,1Fam,5,1999,2000,VinylSd,0.0,953.0,175000.0
1456,20,RL,13175,Inside,1Fam,6,1978,1988,Plywood,163.0,1542.0,210000.0
1457,70,RL,9042,Inside,1Fam,9,1941,2006,CemntBd,0.0,1152.0,266500.0
1458,20,RL,9717,Inside,1Fam,6,1950,1996,MetalSd,1029.0,1078.0,142125.0


In [35]:
# SOME COLUMNS MAY CONTAIN FAKE MISSING VALUES SUCH AS
# "NONE", "NULL", "NA", "?", EMPTY STRING OR SPACE.
# THESE ARE NOT REAL NaN VALUES, SO FIRST WE ARE
# CONVERTING ALL SUCH FAKE MISSING VALUES INTO NaN.
# AFTER THAT, WE ARE REPLACING NaN VALUES USING
# DIFFERENT STRATEGIES BASED ON COLUMN TYPE.

import numpy as np

# LIST OF FAKE MISSING VALUES
fake_missing = ['none', 'null', 'na', '?', '']

# GETTING ALL CATEGORICAL COLUMNS
cat_cols = train_df.select_dtypes(include=['object']).columns

# GETTING ALL NUMERIC COLUMNS
num_cols = train_df.select_dtypes(include=['number']).columns

# HANDLING CATEGORICAL COLUMNS:
# 1. REMOVE EXTRA SPACES
# 2. CONVERT TEXT TO LOWER CASE
# 3. CONVERT FAKE MISSING VALUES TO NaN
# 4. REPLACE NaN WITH MOST FREQUENT VALUE (MODE)

train_df[cat_cols] = (
    train_df[cat_cols]
    .apply(lambda col: col.str.strip().str.lower())
    .replace(fake_missing, np.nan)
    .fillna(train_df[cat_cols].mode().iloc[0])
)

# HANDLING NUMERIC COLUMNS:
# REPLACE NaN VALUES WITH MEDIAN VALUE
# (MEDIAN IS USED TO AVOID THE EFFECT OF OUTLIERS)

train_df[num_cols] = train_df[num_cols].fillna(
    train_df[num_cols].median()
)

# CHECK FOR ANY REMAINING MISSING VALUES
train_df.isnull().sum()


MSSubClass      0
MSZoning        0
LotArea         0
LotConfig       0
BldgType        0
OverallCond     0
YearBuilt       0
YearRemodAdd    0
Exterior1st     0
BsmtFinSF2      0
TotalBsmtSF     0
SalePrice       0
dtype: int64

In [36]:

#To see percentage of missing value

train_df.isnull().mean() * 100

MSSubClass      0.0
MSZoning        0.0
LotArea         0.0
LotConfig       0.0
BldgType        0.0
OverallCond     0.0
YearBuilt       0.0
YearRemodAdd    0.0
Exterior1st     0.0
BsmtFinSF2      0.0
TotalBsmtSF     0.0
SalePrice       0.0
dtype: float64